In [36]:
import pandas as pd

# Load the data
path = '/data4/msc23104470/version1_data-matrix-TTD-w-HUGO-symbols-ICD-11_RD_encoded.csv'
df = pd.read_csv(path, sep=',')

In [39]:
#copy rid column and append in the end of the dataframe
column_to_copy = df['rid'].copy()

In [40]:
df['RID']=column_to_copy

In [42]:
# Subset the features data for outlier detection
#X = df.iloc[:, 1:979]
factor = 3

# Initialize a DataFrame to store columns that do not contain outliers
filtered_data = pd.DataFrame()

# Loop through each column in the selected range
for column in X.columns[1:979]:
    # Calculate the mean and standard deviation of the column
    col_mean = X[column].mean()
    col_std = X[column].std()
    
    # Calculate the upper and lower limits
    upper_lim = col_mean + col_std * factor
    lower_lim = col_mean - col_std * factor
    
    # Filter the column to remove outliers
    filtered_col = X[(X[column] <= upper_lim) & (X[column] >= lower_lim)][column]
    
    # Append the filtered column to the filtered_data DataFrame
    filtered_data[column] = filtered_col

# After filtering, you may have different row indices, so resetting the index is often useful
filtered_data.reset_index(drop=True, inplace=True)

# Subset the columns to keep that are not being filtered for outliers
non_feature_columns = df.iloc[:, 979:]

# Concatenate the non-feature columns with the filtered feature data
final_data = pd.concat([filtered_data, non_feature_columns], axis=1)

# Reset index to ensure consistent indexing
final_data.reset_index(drop=True, inplace=True)

/tmp/ipykernel_16216/3999458932.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  filtered_data[column] = filtered_col
/tmp/ipykernel_16216/3999458932.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  filtered_data[column] = filtered_col
/tmp/ipykernel_16216/3999458932.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use 

In [43]:
# Check for null values in each row of final_data
null_counts_per_row = final_data.isnull().sum(axis=1)

# Find rows where null values exceed 900
rows_to_drop = null_counts_per_row[null_counts_per_row > 900].index

# Drop rows from final_data
final_data_filtered = final_data.drop(rows_to_drop)

# Reset index to ensure consistent indexing
final_data_filtered.reset_index(drop=True, inplace=True)

In [44]:
final_data_filtered.isnull().sum(axis=1)

0       11
1       12
2       12
3       15
4       15
        ..
4023    15
4024    16
4025    15
4026     7
4027    11
Length: 4028, dtype: int64

In [47]:
# move the last column of the dataframe to the first position
# Get the column names as a list
columns = final_data_filtered.columns.tolist()

# Move the last column to the first position
columns = [columns[-1]] + columns[:-1]

# Reorder the DataFrame columns
final_data_filtered = final_data_filtered[columns]

In [49]:
final_data_filtered.rename(columns={'RID': 'rid'}, inplace=True)

In [52]:
final_data_filtered.to_csv('/data4/msc23104470/data-matrix-outlier-removed.csv', index=False)